In [9]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import threading
from PIL import Image, ImageTk
import io
import requests

def load_csv():
    """加载CSV文件并显示数据"""
    try:
        file_path = 'xiaohongshu_posts.csv'
        df = pd.read_csv(file_path)
        show_data(df)
    except Exception as e:
        messagebox.showerror("错误", f"读取 CSV 文件时出错: {e}")

def show_data(dataframe):
    """在Treeview中显示数据"""
    # 清空现有数据
    for row in tree.get_children():
        tree.delete(row)
    
    # 设置列
    tree["columns"] = list(dataframe.columns)
    for column in dataframe.columns:
        tree.heading(column, text=column)
        tree.column(column, width=100, stretch=True)
    
    # 调整特定列的宽度
    if "标题" in tree["columns"]:
        title_index = tree["columns"].index("标题")
        tree.column(tree["columns"][title_index], width=300)
    if "正文内容" in tree["columns"]:
        desc_index = tree["columns"].index("正文内容")
        tree.column(tree["columns"][desc_index], width=400)
    
    # 插入数据
    for index, row in dataframe.iterrows():
        tree.insert("", tk.END, values=list(row))

def search_data():
    """根据搜索条件筛选数据"""
    keyword = search_entry.get()
    search_scope = search_scope_combobox.get()
    
    if not keyword:
        messagebox.showinfo("提示", "请输入搜索关键词。")
        return
    
    try:
        df = pd.read_csv('xiaohongshu_posts.csv')
        result_df = pd.DataFrame(columns=df.columns)
        
        for index, row in df.iterrows():
            if search_scope == "标题" and keyword in str(row['标题']):
                result_df = result_df.append(row, ignore_index=True)
            elif search_scope == "作者" and keyword in str(row['作者']):
                result_df = result_df.append(row, ignore_index=True)
            elif search_scope == "标签" and keyword in str(row['标签']):
                result_df = result_df.append(row, ignore_index=True)
        
        if not result_df.empty:
            show_data(result_df)
        else:
            messagebox.showinfo("搜索结果", "未找到匹配的结果。")
    except Exception as e:
        messagebox.showerror("错误", f"搜索时出错: {e}")

def show_details():
    """显示选中行的详细信息"""
    selected_item = tree.selection()
    if not selected_item:
        messagebox.showinfo("提示", "请先选中一行数据。")
        return
    
    values = tree.item(selected_item, 'values')
    cover_image_url = values[6]  # 假设封面图链接是表格中的第七列
    content = values[4]  # 假设正文内容是表格中的第五列
    
    # 创建新窗口显示详细信息
    detail_window = tk.Toplevel(root)
    detail_window.title("详细信息")
    detail_window.geometry("800x600")  # 设置窗口大小
    
    # 创建一个Frame来放置图片和正文
    content_frame = tk.Frame(detail_window)
    content_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
    
    # 左侧图片区域
    image_frame = tk.Frame(content_frame)
    image_frame.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
    
    # 右侧正文区域
    text_frame = tk.Frame(content_frame)
    text_frame.pack(side=tk.RIGHT, expand=True, fill=tk.BOTH)
    
    # 显示图片
    if cover_image_url and cover_image_url != "封面图未找到":
        try:
            response = requests.get(cover_image_url, stream=True)
            if response.status_code == 200:
                img_data = response.content
                img = Image.open(io.BytesIO(img_data))
                img.thumbnail((350, 350))  # 调整图片大小
                img_tk = ImageTk.PhotoImage(img)
                
                img_label = tk.Label(image_frame, image=img_tk)
                img_label.image = img_tk  # 保持引用
                img_label.pack(pady=10, expand=True)
        except Exception as e:
            tk.Label(image_frame, text=f"加载图片时出错: {e}").pack(pady=5)
    else:
        tk.Label(image_frame, text="封面图未找到").pack(pady=5)
    
    # 显示正文内容
    tk.Label(text_frame, text="正文内容:", font=("Arial", 12, "bold")).pack(pady=5)
    text_widget = tk.Text(text_frame, wrap=tk.WORD, width=50, height=20)
    text_widget.insert(tk.END, content)
    text_widget.pack(pady=5, expand=True, fill=tk.BOTH)

def open_link(url):
    """打开链接"""
    import webbrowser
    webbrowser.open(url)

# 创建主窗口
root = tk.Tk()
root.title("小红书数据查看器")

# 创建表格
tree = ttk.Treeview(root, show="headings")
tree.pack(fill=tk.BOTH, expand=True)

# 创建滚动条
vsb = ttk.Scrollbar(root, orient="vertical", command=tree.yview)
hsb = ttk.Scrollbar(root, orient="horizontal", command=tree.xview)
tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
vsb.pack(side=tk.RIGHT, fill=tk.Y)
hsb.pack(side=tk.BOTTOM, fill=tk.X)

# 创建搜索框和按钮
search_frame = tk.Frame(root)
search_frame.pack(fill=tk.X, padx=10, pady=10)

tk.Label(search_frame, text="搜索关键词:").pack(side=tk.LEFT, padx=5)
search_entry = tk.Entry(search_frame)
search_entry.pack(side=tk.LEFT, padx=5)

search_scope_label = tk.Label(search_frame, text="搜索范围:")
search_scope_label.pack(side=tk.LEFT, padx=5)
search_scope_combobox = ttk.Combobox(search_frame, values=["标题", "作者", "标签"])
search_scope_combobox.current(0)  # 默认选择“标题”
search_scope_combobox.pack(side=tk.LEFT, padx=5)

search_button = tk.Button(search_frame, text="搜索", command=search_data)
search_button.pack(side=tk.LEFT, padx=5)

# 添加“查看详细”按钮
detail_button = tk.Button(search_frame, text="查看详细", command=show_details)
detail_button.pack(side=tk.LEFT, padx=5)

# 添加“刷新”按钮
refresh_button = tk.Button(root, text="刷新", command=load_csv)
refresh_button.pack(pady=10)

# 加载数据
load_csv()

# 运行主循环
root.mainloop()